# 2020 Befolkning i Oslo på grunnkrets

In [1]:
import json
import os
import pandas as pd
import requests
import numpy as np
import ipyleaflet

from branca.colormap import linear
from pyjstat import pyjstat

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

Vi heter opp en GeoJSON-fil med polygoner for alle grunnkretsene i Oslo

In [2]:
geodata = json.load(open('data/oslo-grunnkrets.json'))

Vi henter opp spørringen mot Statistikkbanken. 

In [3]:
query = json.load(open('data/befolkning-query.json'))

Spørringen og endepunktet til API-et kan hentes fra Statistikkbanken.

In [4]:
res = requests.post("https://data.ssb.no/api/v0/no/table/04317/", json=query)

In [5]:
ds = pyjstat.Dataset.read(res.text)
befolkning = ds.write('dataframe', naming='id')

Vi fjerner "NaN" fra datasettet og erstatter med "0". Dette er ikke helt riktig, siden disse er fjernet av konfedensialitetshensyn. I tillegg liker ikke Cloropleth-rutinen verdier som ikke er numeriske. Vi printer også ut datastrukturen slik at vi vet hvordan data ser ut.

In [6]:
befolkning.replace(np.NaN, 0, inplace=True)
befolkning

Grunnkretser ContentsCode   Tid  value
0       03010101    Personer1  2020    4.0
1       03010102    Personer1  2020  309.0
2       03010103    Personer1  2020   33.0
3       03010104    Personer1  2020  413.0
4       03010105    Personer1  2020  502.0
..           ...          ...   ...    ...
586     03016003    Personer1  2020    3.0
587     03016004    Personer1  2020   17.0
588     03016005    Personer1  2020   16.0
589     03016006    Personer1  2020    2.0
590     03016007    Personer1  2020  347.0

[591 rows x 4 columns]

Vi bygger opp en mapping mellom grunnkrets-id'ene og verdiene. Disse brukes av Chropleth-rutinen som lager fargekartet. Her går det også an å velge andre basemaps om det er ønskelig. En utfordring med iPyleaflet sin Choropleth-rutine er at den forutsetter at ID-en som verdien skal mappes mot, ligger på feature-nivå i GeoJSON-fila. Dette er ikke alltid tilfellet og krever litt jobbing med kartgrunnlaget.

In [7]:
be = dict(zip(befolkning['Grunnkretser'].tolist(), befolkning['value'].tolist()))


In [8]:
layer = ipyleaflet.Choropleth(
    geo_data=geodata,
    choro_data=be,
    colormap=linear.YlOrRd_04,
    style={'fillOpacity': 0.6, 'dashArray': '1, 1'}
)

Vi velger et senter-punkt + zoom.

In [9]:
center = [59.905695, 10.737880]
zoom = 12

In [10]:
m = Map(center=center, zoom=zoom)

In [11]:
m.add_layer(layer)
m

Map(center=[59.905695, 10.73788], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [12]:
from ipywidgets import Text, HTML
from ipyleaflet import WidgetControl, GeoJSON 

html1 = HTML('''
    <h3>Oslo befolkningstall 2020</h3>
''')
html1.layout.margin = '20px 20px 20px 20px'
control1 = WidgetControl(widget=html1, position='bottomright')
m.add_control(control1)

def update_html(feature, **kwargs):
    html1.value = '''
        <h3>Oslo befolkningstall 2020</h3>
        <h4>{0}</h4>
        {1:.0f} personer
    '''.format(feature['properties']['grunnkretsnavn'], be[feature['properties']['grunnkrets_id']])

layer.on_hover(update_html)